In [1]:
import os
import sys
import math
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torchvision
from tqdm import tqdm_notebook as tqdm, tnrange
from transformers.tokenization_bert import BertTokenizer
from transformers import BertModel
from transformers.optimization import AdamW, get_linear_schedule_with_warmup

w_dir = %pwd
work_dir = os.path.dirname(w_dir)
work_dir

I1210 03:11:43.879435 139856216110912 file_utils.py:39] PyTorch version 1.1.0 available.


'/work'

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.manual_seed(12)
bert_model_name = 'bert-base-chinese'
warmup_proportion = 0.1
learning_rate = 2e-5
num_epochs = 20
eval_frequency = 5
batch_size = 16

In [3]:
sys.path.append(w_dir+'/fgc_support_retri')

In [4]:
from train import train_context_model

I1210 03:11:47.023138 139856216110912 corenlp.py:42] Using an existing server http://140.109.19.191:9000
I1210 03:11:48.026338 139856216110912 corenlp.py:118] The server is available.


In [5]:
train_context_model(5, 16, "test")

I1204 10:35:46.791498 139650299205440 configuration_utils.py:152] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-chinese-config.json from cache at /root/.cache/torch/transformers/8a3b1cfe5da58286e12a0f5d7d182b8d6eca88c08e26c332ee3817548cf7e60a.0c16faba8be66db3f02805c912e4cf94d3c9cffc1f12fa1a39906f9270f76d33
I1204 10:35:46.794856 139650299205440 configuration_utils.py:169] Model config {
  "attention_probs_dropout_prob": 0.1,
  "directionality": "bidi",
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_labels": 2,
  "output_attentions": false,
  "output_hidden_states": false,
  "output_past": true,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "po

no gold supporting evidence
{'QID': 'D001Q11', 'QTYPE': '申论', 'QTEXT': '蘇東坡為何被後人認為是文學藝術史上的通才?', 'SENTS': [{'text': '苏东坡为何被后人认为是文学艺术史上的通才?', 'start': 0, 'end': 21}], 'ANSWER': [{'ATEXT': '', 'ATOKEN': [{'text': '', 'start': 0}], 'ATEXT_CN': ''}], 'ATYPE': 'Event', 'AMODE': 'Single-Span-Extraction', 'ASPAN': [], 'SHINT': [], 'QTEXT_CN': '苏东坡为何被后人认为是文学艺术史上的通才?'}
no gold supporting evidence
{'QID': 'D006Q02', 'QTYPE': '申论', 'QTEXT': '「阿拉伯之春」運動中，走上街頭的民眾的訴求為何?', 'SENTS': [{'text': '「阿拉伯之春」运动中，', 'start': 0, 'end': 11}, {'text': '走上街头的民众的诉求为何?', 'start': 11, 'end': 24}], 'ANSWER': [{'ATEXT': '', 'ATOKEN': [{'text': '', 'start': 0}], 'ATEXT_CN': ''}], 'ATYPE': 'Object', 'AMODE': 'Single-Span-Extraction', 'ASPAN': [], 'SHINT': [], 'QTEXT_CN': '「阿拉伯之春」运动中，走上街头的民众的诉求为何?'}
no gold supporting evidence
{'QID': 'D048Q09', 'QTYPE': '申论', 'QTEXT': '聊天機器人仰賴哪些方法讓回答愈來愈準確?', 'SENTS': [{'text': '聊天机器人仰赖哪些方法让回答愈来愈准确?', 'start': 0, 'end': 20}], 'ANSWER': [{'ATEXT': '', 'ATOKEN': [{'text': '', 'start': 0}], 'A

I1204 10:35:55.083246 139650299205440 tokenization_utils.py:375] loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-chinese-vocab.txt from cache at /root/.cache/torch/transformers/8a0c070123c1f794c42a29c6904beb7c1b8715741e235bee04aca2c7636fc83f.9b42061518a39ca00b8b52059fd2bede8daa613f8a8671500e518a8c29de8c00
  0%|          | 0/47 [00:00<?, ?it/s]

start training ... 


100%|██████████| 47/47 [00:42<00:00,  1.67it/s]


...

epoch 3 train_loss: 0.177


  0%|          | 0/47 [00:00<?, ?it/s]

epoch 4 train_loss: 0.176


100%|██████████| 47/47 [00:34<00:00,  1.68it/s]


epoch 5 train_loss: 0.177
epoch 5 eval_loss: 0.121


In [15]:
import config
from utils import read_fgc
from fgc_preprocess import SerContextDataset, BertSpanIdx, bert_collate, bert_context_collate
from sup_model import BertForMultiHopQuestionAnswering, BertSupSentClassification

In [16]:
items = read_fgc(config.FGC_DEV)

no gold supporting evidence
{'QID': 'D032Q10', 'QTYPE': '进阶题', 'QTEXT': '第二次簽訂的北美貿易協定從簽署至生效過了幾日?', 'SENTS': [{'text': '第二次签订的北美贸易协定从签署至生效过了几日?', 'start': 0, 'end': 23}], 'ANSWER': [{'ATEXT': '資訊不足無法判定', 'ATOKEN': [{'text': '资讯不足无法判定', 'start': -1}], 'ATEXT_CN': '资讯不足无法判定'}], 'ATYPE': 'Date-Duration', 'AMODE': 'Date-Duration', 'ASPAN': [], 'SHINT': [], 'QTEXT_CN': '第二次签订的北美贸易协定从签署至生效过了几日?'}
no gold supporting evidence
{'QID': 'D049Q04', 'QTYPE': '申论', 'QTEXT': '「雅婷逐字稿」的命名起源為何?', 'SENTS': [{'text': '「雅婷逐字稿」的命名起源为何?', 'start': 0, 'end': 15}], 'ANSWER': [{'ATEXT': '', 'ATOKEN': [{'text': '', 'start': 0}], 'ATEXT_CN': ''}], 'ATYPE': 'Event', 'AMODE': 'Single-Span-Extraction', 'ASPAN': [], 'SHINT': [], 'QTEXT_CN': '「雅婷逐字稿」的命名起源为何?'}
no gold supporting evidence
{'QID': 'D086Q03', 'QTYPE': '申论', 'QTEXT': '不可再生能源的意義是什麼？', 'SENTS': [{'text': '不可再生能源的意义是什么？', 'start': 0, 'end': 13}], 'ANSWER': [{'ATEXT': '', 'ATOKEN': [{'text': '', 'start': 0}], 'ATEXT_CN': ''}], 'ATYPE': 'Object', 'AMODE': 'Sing

In [17]:
len(items)

213

In [18]:
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')
train_set = SerContextDataset(items, transform=torchvision.transforms.Compose([BertSpanIdx(tokenizer)]))
dataloader_train = DataLoader(train_set, batch_size=4, collate_fn=bert_context_collate)

I1204 10:08:58.525778 140007975401280 tokenization_utils.py:375] loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-chinese-vocab.txt from cache at /root/.cache/torch/transformers/8a0c070123c1f794c42a29c6904beb7c1b8715741e235bee04aca2c7636fc83f.9b42061518a39ca00b8b52059fd2bede8daa613f8a8671500e518a8c29de8c00


In [19]:
bert_encoder = BertModel.from_pretrained('bert-base-chinese')
model = BertForMultiHopQuestionAnswering(bert_encoder)

model.to(device)
if n_gpu > 1:
    model = nn.DataParallel(model)

param_optimizer = list(model.named_parameters())

no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

I1204 10:08:59.440470 140007975401280 configuration_utils.py:152] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-chinese-config.json from cache at /root/.cache/torch/transformers/8a3b1cfe5da58286e12a0f5d7d182b8d6eca88c08e26c332ee3817548cf7e60a.0c16faba8be66db3f02805c912e4cf94d3c9cffc1f12fa1a39906f9270f76d33
I1204 10:08:59.445090 140007975401280 configuration_utils.py:169] Model config {
  "attention_probs_dropout_prob": 0.1,
  "directionality": "bidi",
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_labels": 2,
  "output_attentions": false,
  "output_hidden_states": false,
  "output_past": true,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "po

In [20]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
num_train_optimization_steps = int(math.ceil(len(train_set) / batch_size)) * num_epochs
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps=int(num_train_optimization_steps * warmup_proportion),
                                            num_training_steps=num_train_optimization_steps)

In [21]:
for batch in tqdm(dataloader_train):
    input_ids = batch['input_ids'].to(device)
    token_type_ids = batch['token_type_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    assert input_ids.shape == token_type_ids.shape == attention_mask.shape
    if input_ids.shape[1] > 512:
        print(input_ids.shape)

In [23]:
for epoch_i in tnrange(5 + 1):
    model.train()
    running_loss = 0.0
    for batch_i, batch in enumerate(tqdm(dataloader_train)):
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        token_type_ids = batch['token_type_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)
        labels.type(torch.long)

        loss = model(input_ids,
                    token_type_ids=token_type_ids,
                     attention_mask=attention_mask,
                     mode=BertForMultiHopQuestionAnswering.ForwardMode.TRAIN,
                     se_start_labels=labels)

        if n_gpu > 1:
            loss = loss.mean()  # mean() to average on multi-gpu.

        loss.backward()
        optimizer.step()
        scheduler.step()
        running_loss += loss.item()

    print('epoch %d train_loss: %.3f' % (epoch_i, running_loss / len(dataloader_train)))

epoch 0 train_loss: 0.106


epoch 1 train_loss: 0.081


epoch 2 train_loss: 0.080


epoch 3 train_loss: 0.077


epoch 4 train_loss: 0.081


epoch 5 train_loss: 0.079


In [14]:
loss